In [51]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
import pandas as pd
import numpy as np
import importlib
import pymysql
import requests

from db_manager import db_manager

In [29]:
#2023년의 모든 날짜를 생성하여 db에 넘겨주기
#날짜 생성은 넘파이에서? 판다스인가? 여튼.. 그 후 DB연결하고 data 쓰기.. db 모듈을 만들어야 할 듯? 우선 여기의 하부 디렉토리에 만든 후 모듈로 독립적으로 사용하도록 하자. 
index_date = pd.date_range('2023/01/01', periods=365)
type(index_date)

pandas.core.indexes.datetimes.DatetimeIndex

In [85]:
importlib.reload(db_manager)
dbm = db_manager.DbManager()

<module 'db_manager.db_manager' from 'c:\\Users\\jchoi\\Coding\\python\\pocketmon\\db_manager\\db_manager.py'>

Exception ignored in: <function DbManager.__del__ at 0x000001D05A38BCE0>
Traceback (most recent call last):
  File "c:\Users\jchoi\Coding\python\pocketmon\db_manager\db_manager.py", line 50, in __del__
    self.close()
  File "c:\Users\jchoi\Coding\python\pocketmon\db_manager\db_manager.py", line 59, in close
    self.conn.close()
  File "c:\Users\jchoi\anaconda3\envs\py311\Lib\site-packages\pymysql\connections.py", line 404, in close
    raise err.Error("Already closed")
pymysql.err.Error: Already closed


Connected


In [84]:
dbm.close()

In [86]:
dbm.execute("show tables")
dbm.print_fetched()

('pokemon_evol_tb',)
('pokemon_info_tb',)


In [26]:
#이제 날짜를 넣어보자. 
sql = r'insert into date_tb (date_in_day) values (%s)'
for d in index_date:
    dbm.execute(sql, (d))

In [ ]:
dbm.execute("select * from date_tb")
dbm.print_fetched()

In [32]:
data = np.arange(12)
data = data.reshape(4, 3)
index_data = pd.date_range('2023/01/01', periods=4)
columns_data = ['a', 'b', 'c']

pd.DataFrame(data, index=index_data, columns=columns_data)

,a,b,c
2023-01-01,0,1,2
2023-01-02,3,4,5
2023-01-03,6,7,8
2023-01-04,9,10,11


In [ ]:
url = 'https://pokemongo.inven.co.kr/dataninfo/pokemon/'
response = requests.get(url)

if response.status_code == 200:  # Check if request was successful
    print(response.text)  # Print the content of the webpage
else:
    print('Error:', response.status_code)

In [89]:
from bs4 import BeautifulSoup

pokemon_soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
from cgitb import text


pokemons_soup = pokemon_soup.select('#pokemongoList > ul > li > a')

pokemon_number_list = []
pokemon_name_list = []
pokemon_type1_list = []
pokemon_type2_list = []
pokemon_type1_src = []
pokemon_type2_src = []

for p in pokemons_soup:
    poke_text = p.select('span.pokemonname')[0].text
    #번호와 이름을 분리한다. 
    num_and_name = poke_text.split('.')
    print(num_and_name)
    
    pokemon_number_list.append(num_and_name[0].strip())
    pokemon_name_list.append(num_and_name[1].strip())
    pokemon_type1_src.append()


In [110]:
pokemon_number_list[0]
pokemon_name_list[0]

'1'

'이상해씨'

In [21]:
pocketmon_names = soup.find_all('span', {'class':'pokemonname'})
for p in pocketmon_names:
    pass
    # print(p.text)
pocketmon = p.parent

In [75]:
# url = 'https://pokemongo.inven.co.kr/'+pocketmon['href']
url = 'https://pokemongo.inven.co.kr/dataninfo/pokemon/detail.php?code=133'
pocketmon_info_html = requests.get(url)

In [76]:
pocketmon_info_soup = BeautifulSoup(pocketmon_info_html.text, 'html.parser')

In [52]:
pocketmon_info_soup.select()

<li class="item"><a class="link" href="https://www.inven.co.kr/">웹진</a></li>

In [77]:
evol_list = pocketmon_info_soup.find('li', {'class':'arrow'}).parent
lists = evol_list.find_all('li')

In [81]:
# lists
evol_step = 0
cur_name = ''
par_name = ''

for l in lists:
    get_by_class = l.get('class')
    if get_by_class != None:
        print(get_by_class[0])
        evol_step += 1
        par_name = cur_name
    else:
        cur_name = l.find('span').find('a').text
        if par_name != '':
            print(f"parent: {par_name}, and Name: {cur_name}")            
        else:
            print(f"no parent. Name: {cur_name}")
        
evol_step    

no parent. Name: 133.이브이
arrow
parent: 133.이브이, and Name: 134.샤미드
parent: 133.이브이, and Name: 135.쥬피썬더
parent: 133.이브이, and Name: 136.부스터
parent: 133.이브이, and Name: 196.에브이
parent: 133.이브이, and Name: 197.블래키
parent: 133.이브이, and Name: 470.리피아
parent: 133.이브이, and Name: 471.글레이시아


1